In [ ]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

In [ ]:
from google.colab import userdata
Open_Router_ApiKey = userdata.get('nano_open_router_api')

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key= Open_Router_ApiKey
)

**API Calling and Processing Pipeline**

In [ ]:
import requests
import json
import base64
from pathlib import Path

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {Open_Router_ApiKey}",
    "Content-Type": "application/json"
}

# Read and encode the image
image_path = "/content/base_image.jpeg"
base64_image = encode_image_to_base64(image_path)
base_image_url = f"data:image/jpeg;base64,{base64_image}"

# Read and encode the image
image_path = "/content/without_bg_bed.png"
base64_image = encode_image_to_base64(image_path)
overlay_image_url_1 = f"data:image/jpeg;base64,{base64_image}"

# Read and encode the image
image_path = "/content/without_bg_sofa.jpg"
base64_image = encode_image_to_base64(image_path)
overlay_image_url_2 = f"data:image/jpeg;base64,{base64_image}"

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": ""
            },
            {
                "type": "image_url",
                "image_url": {"url": base_image_url}
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "I'd like to see the room furnished with a bed, a sofa, plants, rugs and table lamps but use these images of bed I’m attaching here"
            },
            {
                "type": "image_url",
                "image_url": {"url": overlay_image_url_1}
            },
            {
                "type": "image_url",
                "image_url": {"url": overlay_image_url_2}
            }
        ]
    }
]

payload = {
    "model": "google/gemini-2.5-flash-image-preview:free",
    "messages": messages
}

response = requests.post(url, headers=headers, json=payload)
print(response.json())

In [ ]:
import base64

# Convert Response object to Python dict
resp_json = response.json()

# Extract the full data URL
data_url = resp_json["choices"][0]["message"]["images"][0]["image_url"]["url"]

# Split metadata from the base64 string
header, base64_data = data_url.split(",", 1)

# Decode base64 into bytes
image_bytes = base64.b64decode(base64_data)

# Save as PNG
with open("output_5.png", "wb") as f:
    f.write(image_bytes)

print("Image saved as output.png")

**Test Code with Change Imput Prompt**

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": ""
            },
            {
                "type": "image_url",
                "image_url": {"url": base_image_url}
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "I'd like to see the room furnished with a bed, a sofa, plants, rugs and table lamps Put bed to the left side of room but use these images of bed I’m attaching here"
            },
            {
                "type": "image_url",
                "image_url": {"url": overlay_image_url_1}
            },
            {
                "type": "image_url",
                "image_url": {"url": overlay_image_url_2}
            }
        ]
    }
]

payload = {
    "model": "google/gemini-2.5-flash-image-preview:free",
    "messages": messages
}

response = requests.post(url, headers=headers, json=payload)
print(response.json())

**Code Ends**

In [ ]:
"""
# 1. Create a session (one-time)
session_payload = {
    "model": "google/gemini-2.5-flash-image-preview:free"
}
session_res = requests.post(session_url, headers=headers, json=session_payload).json()
session_id = session_res["name"]  # unique session name

# 2. First call: send base image
payload1 = {
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Here is the base image."},
                {"type": "image_url", "image_url": {"url": base_image_url}}
            ]
        }
    ]
}
res1 = requests.post(f"{session_url}/{session_id}:sendMessage",
                     headers=headers, json=payload1).json()

# 3. Second call: send new prompt + objects (Gemini remembers base image automatically)
payload2 = {
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Now add these objects to the base image."},
                {"type": "image_url", "image_url": {"url": overlay_image_url_1}},
                {"type": "image_url", "image_url": {"url": overlay_image_url_2}}
            ]
        }
    ]
}
res2 = requests.post(f"{session_url}/{session_id}:sendMessage",
                     headers=headers, json=payload2).json()
"""

**Multiple Tool Calls**

In [ ]:
{
  "model": "anthropic/claude-3.5-sonnet",
  "messages": [
    {
      "role": "user",
      "content": "Research the environmental impact of electric vehicles and provide a comprehensive analysis."
    }
  ],
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "search_academic_papers",
        "description": "Search for academic papers on a given topic",
        "parameters": {
          "type": "object",
          "properties": {
            "query": {"type": "string"},
            "field": {"type": "string"}
          },
          "required": ["query"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_latest_statistics",
        "description": "Get latest statistics on a topic",
        "parameters": {
          "type": "object",
          "properties": {
            "topic": {"type": "string"},
            "year": {"type": "integer"}
          },
          "required": ["topic"]
        }
      }
    }
  ]
}


**A Simple Agentic Loop**

In [ ]:

def call_llm(msgs):
    resp = openai_client.chat.completions.create(
        model=google/gemini-2.0-flash-001,
        tools=tools,
        messages=msgs
    )
    msgs.append(resp.choices[0].message.dict())
    return resp

def get_tool_response(response):
    tool_call = response.choices[0].message.tool_calls[0]
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)

    # Look up the correct tool locally, and call it with the provided arguments
    # Other tools can be added without changing the agentic loop
    tool_result = TOOL_MAPPING[tool_name](**tool_args)

    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": tool_result,
    }

max_iterations = 10
iteration_count = 0

while iteration_count < max_iterations:
    iteration_count += 1
    resp = call_llm(_messages)

    if resp.choices[0].message.tool_calls is not None:
        messages.append(get_tool_response(resp))
    else:
        break

if iteration_count >= max_iterations:
    print("Warning: Maximum iterations reached")

print(messages[-1]['content'])

In [ ]:
"""
import os
from google.colab import drive
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get("nano_api")

drive.mount("/content/drive")

# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
os.chdir("/content/drive/MyDrive/Google AI Studio")
"""

**Other Supported Functions of NanoBanana**

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('nano_api')

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
MODEL_ID = "gemini-2.5-flash-image-preview"

In [ ]:
from IPython.display import display, Markdown, Image
import pathlib

# Loop over all parts and display them either as text or images
def display_response(response):
  for part in response.parts:
    if part.text:
      display(Markdown(part.text))
    elif image:= part.as_image():
      display(image)
      # image.show() if not in a notebook

# Save the image
# If there are multiple ones, only the last one will be saved
def save_image(response, path):
  for part in response.parts:
    if image:= part.as_image():
      image.save(path)

**Cont. Chat Editing**

In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
)

In [ ]:
message = "create a image of a plastic toy fox figurine in a kid's bedroom, it can have accessories but no weapon" # @param {type:"string"}

response = chat.send_message(message)
display_response(response)
save_image(response, "figurine.png")

In [ ]:
message = "Add a blue planet on the figuring's helmet or hat (add one if needed)" # @param {type:"string"}
response = chat.send_message(message)
display_response(response)